## What version are we?

In [ ]:
#!about

In [ ]:
$psversionTable

In [ ]:
$host.Name
$host.UI.RawUI

## Markdown cells contain formatted text and images 

* *emphasis*
* **Strong Emphasis**
* `code`
* etc. 

```
 Write-Host "This is a script block formatted as PowerShell" -fore red
 # How good the format is varies ! 
```

**Folding** markdown cells hides the code cells between them and the next markdown with the same heading level.   
**Warning** \# signs at the start of a line in a code will be misinterpreted by the folding algorithm as heading markers, so indent them as a work-round.  

Local images work better if the path is relative to the .ipynb file or they can be fetched over http    
![Untitled2.png](./Untitled2.png)    
![James Avatar](https://jhoneill.github.io/assets/james.jpg)

It is also possible to have code cells which contain markdown or html... 

In [ ]:
<strong>.NET interactive allows Cells to have a <em>language</em> of 'HTML'</strong><br/>  
Language is selected with <em>Magic commands</em> <code>#!html</code>, <code>#!pwsh</code>, 
<code>#!javascript</code>, <code>#!fsharp</code> and <code>#!csharp</code>. <br/>
There is a full list of magic commands further down.

In [ ]:
#!markdown
**Or a language of _Markdown_**

## We can do normal shell / command-line things ... 
Note: some commands don't like the `.NET interactive` Kernel in VSCode, but will run if you select the `.NET (PowerShell)` Kernel

In [ ]:
nslookup google.com

In [ ]:
write-host -ForegroundColor Green "boo hoo"

## Where were we called from ?

In [ ]:
$pshome
$p = Get-process -id $pid   
$p, $p.Parent, $p.Parent.Parent,$p.Parent.Parent.parent,$p.Parent.Parent.parent.parent
Get-CimInstance -ClassName win32_process -Filter "processID = $($p.id)" |  select -expand commandline
Get-CimInstance -ClassName win32_process -Filter "processID = $($p.Parent.id)" |  select -expand commandline
Get-CimInstance -ClassName win32_process -Filter "processID = $($p.Parent.parent.id)" |  select -expand commandline
Get-CimInstance -ClassName win32_process -Filter "processID = $($p.Parent.parent.parent.id)" |  select -expand commandline
Get-CimInstance -ClassName win32_process -Filter "processID = $($p.Parent.parent.parent.parent.id)" |  select -expand commandline

*In a browser you can check who owns the port you connected to*   
Note: things which display a progress bar and then try to close it caused errors, so we turned progress display off. This seems to be fixed now :-) 

In [ ]:
# $ProgressPreference = 'silentlyContinue'
Get-NetTCPConnection -LocalPort 9815,8888 -State Listen | ft -a

## PowerShell modules loaded in a clean session, and modules available

In [ ]:
Get-module

In [ ]:
Get-Module -ListAvailable | 
    group -no -Property @{e={
        Split-Path -Parent $_.Path | 
            %{if ([Version]::TryParse((Split-Path $_ -Leaf), [ref]$null)) {Split-Path -Parent $_} else {$_} } | 
                 Split-Path -Parent
     }} | ft -AutoSize -wrap  

In [ ]:
Get-Module -ListAvailable | where path -Match "interactive" | ft name ,exportedCommands 

## What is in the PowerShell module that comes with .Net Interactive ? 

In [ ]:
ipmo Microsoft.DotNet.Interactive.PowerShell
Get-command -Module Microsoft.DotNet.Interactive.PowerShell

## Out-Display & preformed HTML
First magic command  `#!time` and use of  `[Microsoft.DotNet.Interactive.Kernel]::HTML($html)` 

In [ ]:
#!time 
$x = "Counting" | Out-Display -PassThru
1..5 | %{sleep 1; $x.Update($_) }

In [ ]:
$html = Get-Command -Module Microsoft.DotNet.Interactive.PowerShell | 
        ConvertTo-Html -Fragment -Property Name , Version 

[Microsoft.DotNet.Interactive.Kernel]::HTML($html) | Out-Display

**If you don't have the files for the next code-cell skip it and load the SVG data from the following one**

In [ ]:
Import-Module ~\documents\azdo\serverautomationdemo\utils\PSGraph\ -Force # My version!
. ~\documents\azdo\serverautomationdemo\utils\PlotDevopsPipelines.ps1

$svg = plot_pipeline ~\documents\azdo\serverautomationdemo\CI\FollowOn.pipeline.yml  -DestinationPath "" 
$svg | out-file -Encoding utf8 -path ./demo.svg

**If you used the previous code-cell you can skip the next one**

In [ ]:
$svg = Get-Content ./demo.svg

In [ ]:
[Microsoft.DotNet.Interactive.Kernel]::HTML($svg) | Out-Display

## My wrapper for Out-Display

In [ ]:
. .\NotebookOutput.ps1
Out-cell -? | select -ExpandProperty Description | Out-String

In [ ]:
out-cell -?

In [ ]:
# Pipe in , convert to table.
Get-command -Module  Microsoft.DotNet.Interactive.PowerShell 
    | Out-Cell -AsTable -Property name,version

In [ ]:
#Pass script block, format as list
Cell -AsList -Property name,version {
    Get-command -Module  Microsoft.DotNet.Interactive.PowerShell 
}

In [ ]:
#Relies on earlier cell to set $svg - pass raw output in a variable  
if ($svg) {
    cell $svg
}

In [ ]:
#If we loaded the grpah module earlier, raw output from a script block.
if (test-path Function:\plot_pipeline) {
    cell  {plot_pipeline ~\documents\azdo\serverautomationdemo\CI\FollowOn.pipeline.yml  -DestinationPath "" }
}

There wrapper has its own more notebook friendly implementation of `Write-Progress` and can output a collapsible view, these can be nested to give a complex tree structure 

In [ ]:
Get-command -Module  Microsoft.DotNet.Interactive.PowerShell  |  Out-TreeView -Title "Commands in the module" -Property Name,Version | out-cell

## Doug Finke's Notebook module

In [ ]:
ipmo PowerShellNotebook
$ProgressPreference = 'silentlyContinue'
Get-command -Module PowerShellNotebook | Get-help | ft -a -wrap name,Synopsis

In [ ]:
Get-Notebook '.\Git tutorial.ipynb'
 
Get-NotebookContent '.\Git tutorial.ipynb' | Select -first 5 | ft 

## Mixed languages, even in a single block, & variable sharing via magic commands
[Details](https://github.com/dotnet/interactive/blob/main/docs/variable-sharing.md)

In [ ]:
Write-Output "Hello from PowerShell!"

#!fsharp
let numbers = [0 .. 5]
numbers

In [ ]:
#!share  --from fsharp numbers
$numbers

In [ ]:
#!value --name someJson
{
    "what": "Some Json",
    "why": "To share it!"
}

In [ ]:
#!share someJson --from value
ConvertFrom-Json -InputObject $someJson

## List of Magic commands

In [ ]:
#!lsmagic